In [2]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import os
import json
import re
import fasttext
import time
from jamo import h2j, j2hcj
from konlpy.tag import Mecab
from gensim.test.utils import common_texts
import pickle

from gensim.models.fasttext import load_facebook_model
from gensim.models.doc2vec import TaggedDocument

from sklearn.metrics import pairwise_distances

from scipy import stats as st

import sys
sys.path.append('/Users/nogyeongju/Desktop/Git/Lionlp')

from cluster import *
from FDoc import *
from util import *
from preprocessing import *
from sklearn.neighbors import NearestNeighbors



In [3]:
'''
from gensim.test.utils import datapath
model = load_facebook_model(datapath("/Users/nogyeongju/Desktop/petition/petition_model.bin"))
sent = [['ㄷㅏ-ㅅㅣ-', 'ㅎㅏㄴㅂㅓㄴ', 'ㅎㅏㄱㅅㅡㅂㅇㅡㄹ', 'ㅎㅏㅂㅅㅣ-ㄷㅏ-'], ['ㅈㅔ-ㅂㅏㄹ', 'ㄷㅗㅣㄱㅣ-ㄹㅡㄹ', 'ㅂㅣㄹㅇㅓ-', 'ㅠ']]
model.build_vocab(sent, update=True)
model.train(sentences=sent, total_examples = len(sent), epochs=5)
'''

'\nfrom gensim.test.utils import datapath\nmodel = load_facebook_model(datapath("/Users/nogyeongju/Desktop/petition/petition_model.bin"))\nsent = [[\'ㄷㅏ-ㅅㅣ-\', \'ㅎㅏㄴㅂㅓㄴ\', \'ㅎㅏㄱㅅㅡㅂㅇㅡㄹ\', \'ㅎㅏㅂㅅㅣ-ㄷㅏ-\'], [\'ㅈㅔ-ㅂㅏㄹ\', \'ㄷㅗㅣㄱㅣ-ㄹㅡㄹ\', \'ㅂㅣㄹㅇㅓ-\', \'ㅠ\']]\nmodel.build_vocab(sent, update=True)\nmodel.train(sentences=sent, total_examples = len(sent), epochs=5)\n'

In [4]:
#model.wv.similarity('ㄷㅐ-ㅌㅗㅇㄹㅕㅇ','ㅁㅜㄴㅈㅐ-ㅇㅣㄴ')

In [5]:
from petitions_dataset import Petitions
petitions = Petitions(begin_date='2017-01-20', end_date='2020-07-30')

In [6]:
petitions.set_keys('content')

In [7]:
#petitions_list = []
petitions_text = []
for content in petitions:
#    petitions_list.append({
#        'id':idx,
#        'begin':begin,
#        'title':title,
#        'content':content,
#        'category':category,
#        'agree':num_agree,
#    })
    petitions_text.append(content)

In [9]:
tmp = pd.DataFrame()
tmp['data'] = petitions_text

In [11]:
tmp.to_csv('./petitions_text.txt')

In [ ]:
model = fasttext.train_unsupervised('petitions_text.txt')

In [20]:
model.save_model('petitions_word_model.bin')

In [8]:
model = fd2v(model_path = './petitions_word_model.bin')

In [2]:
model = fd2v()

In [10]:
#batch_size = 50000

# 용량이 엄청나게 크다. 말도 안되게
#for idx in range(0,len(petitions_text),batch_size):
#    temp = np.array([sentence_to_jamo(text) for text in petitions_text[idx:idx+batch_size]])
#    np.save('./petition_jamo_array'+str(idx)+'.npy',temp)

OSError: 50000 requested and 4852 written

In [ ]:
#data =  [TaggedDocument(text.split(' '), 'tag : #'+str(idx) ) for idx,text in enumerate(petitions_text)]

In [3]:
temp = []
batch_size = 30000
for idx in range(0,len(petitions_text),batch_size):
    for text in petitions_text[idx:idx+batch_size]:
        vec = np.array([model.get_word_vectors(x) for x in text]).mean(axis=0)
        if (type(text) != str or vec.shape == ()):
            print(text)
            temp.append(np.array([0]))
            continue
    #            all_vector.append(np.array([self.model.get_word_vector(x) for x in text]).mean(axis=0))
        temp.append(vec)
    np.save('./petition_vec'+str(idx)+'.npy',temp)

NameError: name 'petitions_text' is not defined

In [22]:
fd2v_

In [9]:
model.fit(petitions_text)

/Users/nogyeongju/Desktop/Git/Lionlp/FDoc/fd2v.py:36: RuntimeWarning: Mean of empty slice.
  vec = np.array([self.model.get_word_vector(x) for x in text]).mean(axis=0)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
model.fit_knn_docs()

In [13]:
input_ = "청소년들의 범죄"
#model.get_document_vector(input_)
#input_ = sentence_to_jamo(input_)
model.get_knn_docs(input_)

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)


{'distance': array([[0.41203612, 0.42030394, 0.4212376 , 0.42178512, 0.4224677 ,
         0.42316353, 0.43128324, 0.43642706, 0.43715572, 0.43752843]],
       dtype=float32),
 'indices': array([[ 79739,  63776, 116940,  22991,  87623, 102335,  22952, 187072,
          18268, 116996]])}

In [10]:
input_= '청소년'
model.model.get_nearest_neighbors(input_)

[(0.9044303894042969, '청소년및'),
 (0.8942788243293762, '청소년폭력'),
 (0.8893114328384399, '청소년용'),
 (0.8861926794052124, '청소년대상'),
 (0.8856853246688843, '청소년용과'),
 (0.8692178726196289, '청소년일'),
 (0.8685765862464905, '청소년'),
 (0.8658358454704285, '청소년증'),
 (0.8623796105384827, '청소년등'),
 (0.8612908720970154, '청소년보호')]

In [29]:
model.get_similar_key_docs(input_,petitions_text)

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

[(-0.53092664, 0.53092664, 223587),
 (-0.52915925, 0.52915925, 300356),
 (-0.47592646, 0.47592646, 64489),
 (-0.5158793, 0.5158793, 90852),
 (-0.4864934, 0.4864934, 364480),
 (-0.3284487, 0.3284487, 2655),
 (-0.46308434, 0.46308434, 244445),
 (-0.51326764, 0.51326764, 273069),
 (-0.45213115, 0.45213115, 299748),
 (-0.47793502, 0.47793502, 391832)]

In [27]:
petitions_text[113]

'안녕하십니까 일베의 만행이 도를 넘어선것같아 알려드리고자 청원을 올립니다. 일베라는 사이트는 연령층의 제한 혹은 로그인없이 누구나 열람을 할수 있는 사이트입니다. 청소년 및 초등학생 어린이도 쉽게 접할수 있고 왜곡된 역사 그리고 성지식을 습득하게 됩니다. 먼저 우리나라 현직 대통령이신 문재인대통령님을 문재앙이라 칭하며 모택동의 사진과 합성하여 게시하며 조롱하고 있고 이러한 차마 형용키 어려운 글들이 수천 수만개는 넘게 있는듯 합니다. 예시링크-> https://www.ilbe.com/9982117937 그리고 돌아가신 김대중대통령님과 노무현대통령님을 합성하며 조롱하는 게시물들은 이루 헤아릴수가 없을 정도이며 많은 수의 추천을 받고 인기글로 오랜기간 게재되고 있습니다. https://www.ilbe.com/9985328249 https://www.ilbe.com/9985969151 또한 왜곡된 성지식을 갖을수 있게 그림으로 만들어 게시하고 있으며 불법 성매매 업소등의 정보를 나누고 있습니다. http://www.ilbe.com/9354714286 http://www.ilbe.com/9416067939 http://www.ilbe.com/8354159159 http://www.ilbe.com/7803181903 http://www.ilbe.com/8343412614 http://www.ilbe.com/7836762341 http://www.ilbe.com/7488007505 간단히 찾은것만도 이정도 입니다. 이러한 글들을 사이트관리자는 수수방관하며 지켜보고만 있고 미성년자를 비롯해 수많은 이들이 접하고 있습니다. 광주민주화운동을 조롱하는 게시물들과 친환경에너지를 개발하고자 하는 정부의 정책에 반대하는 게시물들 역시 수천 수만건이 넘을 정도 입니다. 일일이 게시물링크를 다 넣는것이 불가능 할 정도입니다. 해당사이트는 강력한 제재가 필요하다고 여겨져 민원을 올립니다.'

In [24]:
petitions_text[22952]

'안녕하세요. 저는 경상북도 영덕군 영덕읍에 위치한 영덕야성초등학교 2학년에 재학중인 임대연 학생의 엄마입니다. 제 딸 임대연은 1학년에 입학때부터 2학년인 지금 현재까지 선생님의 학대와 인권침해 그리고 동급생(같은반아이들)들이 보는 앞에서 굉장히 심한 모욕감과 심한 충격을 받을 정도로 선생님들로부터 학생인권을 보호받지 못하였습니다. 그리고 그 동급생들로부터 집단따돌림과 왕따로 학교 생활을 할수 없을 정도로 심리상태도 안좋았으며 학교를 벗어난 학교 밖에서 조차 아이들에게 무시와 모욕감 그리고 왕따로 하여 바깥생활조차도 할수가 없었습니다 . 아이가 학교 등교를 거부하였고 동네에서 놀 친구역시 없어 이사를 하였고 심리상태가 안정이 되지 않아 담임선생님과 학교에 이러한 상황을 여러차례 말씀드렸음에도 불구하고 아이의 인권은 무시하였습니다. 이러한 사안을 바로잡아주시고 아이가 다시금 학교로 돌아가 친구들과 뛰어놀며 정상적으로 학습하고 등교의 즐거움을 배울수 있도록 장관님이 도와주세요. 제발 한가정을 살린다 생각하시고 도와주세요. 아이가 못살면 부모도 함께 못산다는것 생각해주시고 저희 가정을 살려주세요. 제 딸 아이 뿐만아니라 다른 학교 지금 현재 학교에서도 아이를 이렇게 구박하고 학대하는 교사분들이 있는것으로 알고 있습니다. 바로 잡아주세요. 해당 교사의 강력한 처벌을 원합니다. 그리고 이러한 교사가 우리 아이들을 가르치는 자격을 주어서는 안된다고 생각합니다. "왕따가 생기는 이유를 다시 한번 생각해 봐주세요. 왕따가 생기는 이유는 단순히 아이들만의 일로 일어나는것이 아닙니다." 기간 : 2016.3월 중순~ 2017. 현재까지 누가. 1학년 4반 담임. 이진숙선생님(기간제) 2학년 2반 담임. 한미숙선생님(정교사) 저희 아이는 2016년 3월에 영덕야성초등학교에 입학을 하였습니다. 입학당시 한글을 완전히 깨우치지 못하고 입학을 하였습니다. 입학때 담임선생님이 학습이 늦다고 아이를 도움반에 보내고 싶다하셨고 아이가 이상하다고 병원검사를 권유하고 하셨습니다. 그리고 아이

In [58]:
jamo_to_word_sent(petitions_text[285787])

'청원글을 읽다보니, 같은 내용을 중복게재. 동일인이 도배 등 문제점이 많네요. 그 중에서 청원글을 읽다보니 "왜 같은 내용을 계속 올리게 하나?" 의구심이 들고 20만명을 못채우게 하려는 저의(底意)가 숨어있나? 갸우뚱합니다. 청원글을 읽다보니 누군가 같은 내용은 하나로 몰자는 의견도 있으나, 이는 청원자가 할 수 있는 것이 아닌것 같습니다. 읽는 사람 좀 피곤하게 하지 맙시다'

In [5]:
print(fd2v.get_word_vectors(input_).shape)

(5, 100)


In [8]:
fd2v.get_word_vectors(input_)
fd2v.get_word_vectors(petitions_text[0])

array([[ 7.07816958e-01,  6.58292845e-02, -6.96280003e-02,
        -1.20951027e-01, -1.09774642e-01,  3.43723558e-02,
         2.70374924e-01, -4.02612299e-01,  6.06185734e-01,
         4.96389657e-01, -6.31436035e-02,  9.42321494e-02,
         3.14351916e-02,  5.09709977e-02, -3.53998132e-02,
         9.02015418e-02, -5.31322770e-02,  2.05289662e-01,
        -9.67078626e-01,  1.70254529e-01, -1.27407938e-01,
         3.12225729e-01,  5.02560213e-02,  2.90124327e-01,
        -3.64607684e-02, -8.05630386e-01,  8.56059551e-01,
         1.88707903e-01,  8.41165930e-02,  5.61312854e-01,
        -2.94620365e-01, -5.87899089e-01, -5.38229704e-01,
        -9.49922681e-01,  2.79857129e-01,  1.58521220e-01,
         4.52285588e-01,  2.02145446e-02,  1.77164942e-01,
        -5.14185540e-02,  1.20737493e+00,  5.89267910e-01,
        -1.75477982e-01, -2.03434810e-01, -3.43183696e-01,
        -7.15556860e-01,  2.29766190e-01,  2.94449002e-01,
         3.64895195e-01, -2.53296077e-01, -3.26136172e-0

In [9]:
np.min(pairwise_distances(fd2v.get_word_vectors(input_),fd2v.get_word_vectors(petitions_text[0]),metric = 'cosine'),axis=1).sum()
#pairwise_distances(fd2v.get_word_vectors(input_),fd2v.get_word_vectors(petitions_text[0]),metric = 'cosine')

NameError: name 'petitions_text' is not defined

In [53]:
a = fd2v.get_similar_key_docs(input_,petitions_text[0:100])

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encoun

In [54]:
a

array([[-2.1285899e+00,  2.1285899e+00],
       [-1.9111780e+00,  1.9111780e+00],
       [-2.1276386e+00,  2.1276386e+00],
       [-1.6966817e+00,  1.6966817e+00],
       [-1.8501813e+00,  1.8501813e+00],
       [-1.9334682e+00,  1.9334682e+00],
       [-2.0279050e+00,  2.0279050e+00],
       [-3.5762787e-07,  3.5762787e-07],
       [-1.3865285e+00,  1.3865285e+00],
       [-1.3275759e+00,  1.3275759e+00]], dtype=float32)

In [16]:
np.min(fd2v.get_similar_key_docs(X = input_,Y = np.array(['ㅊㅓㅇㄴㅕㄴㅇㅡㄴ ㅇㅓㄷㅣㅇㅔㅅㅓ ㅇㅣㄹㅎㅏㄴㅏㅇㅛ'])), axis=1).sum()

AxisError: axis 1 is out of bounds for array of dimension 0

In [35]:
import heapq

In [49]:
heap = []
heapq.heappush(heap,(-1,1))
heapq.heappush(heap,(-3,3))
heapq.heappush(heap,(-4,4))
heapq.heappush(heap,(-2,2))

In [51]:
heap[0][1]

4

In [54]:
heapq.heappop(heap)

(-2, 2)

In [11]:
fd2v.fit_knn_docs()

In [22]:
fd2v.get_knn_docs(jamo_to_word_sent('장자연씨 사건을 재조사 해야합니다'))

{'distance': array([[0.7123395 , 0.7616435 , 0.78259814, 0.87758577, 0.91233593,
         0.9310685 , 0.94964194, 0.9594879 , 0.9651734 , 0.96538794]],
       dtype=float32),
 'indices': array([[1856,  559, 1769, 2052,  451,  666,   12, 1704,  400,   56]])}

In [34]:
jamo_to_word_sent(petitions_text[1856])

' 국회 패스트트랙 신속처리안건 지정 과정에서 발생한 여야 충돌을 수사하는 경찰이 고심에 잠겼다 오늘 보도된 뉴스입니다 국회가 쓸데없는 정쟁에 휘말려 국가의 중대한 문제들을 해결하는 데 무력한 모습을 보인 지가 이미 수십 년입니다 그 과정 또한 바깥에 알려질까 두려울 정도로 아름답지 못했습니다 여야가 하찮은 정치적 계산에 몰두하여 서로 발목을 잡으며 고성과 욕설이 난무하는 것은 물론이요 몸싸움과 멱살잡이가 일상사였습니다 그 과정에서 민생의 가장 시급한 현안이 끝없이 뒤로 밀려 억울한 희생이 이어진 것이 우리의 현대사입니다 국회의원은 우리의 문제를 해결해달라고 뽑은 우리의 대표들입니다 스스로도 심부름꾼을 자처하는 분들입니다 심부름꾼이 안으로 국민의 피를 빨고 밖으로 나라 망신시키는 꼴을 언제까지 두고 보아야 합니까 국회선진화법은 국민이 강제한 것이 아닙니다 국회의원들 스스로 국민의 대표로서 품위를 지키고 민생의 화급한 현안들을 해결하는 데 집중하자고 합의해서 만든 것 아닙니까 그러나 패스트트랙 법안을 심사하는 과정에서 그들은 다시 동물국회 를 연출하는 추태를 선보였습니다 이제 국민은 더이상 참을 수 없습니다 국회의원 불체포특권을 내세운다는 소릴 들으면 피가 거꾸로 솟는 기분입니다 그런식으로 눙칠 것을 왜 합의해서 법안으로 만든 것입니까 경찰과 검찰은 더이상 한줌도 안 되는 의원들의 눈치를 볼 것 없습니다 어차피 그들은 국민의 지지가 없으면 아무런 권한도 권위도 없는 사람들입니다 수사를 조속히 진행하십시오 그것이 국민의 명령이요 역사의 위임입니다'

In [28]:
fd2v.doc_vecs[0]

[0.07716764509677887,
 0.01679041236639023,
 -0.048659976571798325,
 -0.03601745888590813,
 0.009019524790346622,
 0.012134648859500885,
 -0.09099482744932175,
 -0.06833936274051666,
 -0.08021898567676544,
 -0.02365831471979618,
 0.2098487913608551,
 -0.025388777256011963,
 -0.007539655081927776,
 -0.0027629185933619738,
 0.001971665769815445,
 0.13653722405433655,
 0.020744282752275467,
 -0.047890156507492065,
 0.0111652547493577,
 -0.07661843299865723,
 -0.04129013791680336,
 0.0104160625487566,
 -0.10022099316120148,
 -0.1121060773730278,
 -0.01081994827836752,
 0.09628844261169434,
 -0.0023202572483569384,
 0.07299218326807022,
 -0.03468569368124008,
 0.09838885068893433,
 0.03830955550074577,
 -0.06550160050392151,
 -0.05993771180510521,
 -0.1087953969836235,
 -0.03183876723051071,
 0.07258129864931107,
 0.0832085981965065,
 0.06848641484975815,
 0.03069007210433483,
 0.13486959040164948,
 0.021361403167247772,
 0.11059566587209702,
 0.03251403570175171,
 0.05772245302796364,
 -0.

In [25]:
np.array(np.array([[1,3,4],[3,4,5]]).tolist()).shape

(2, 3)

In [22]:
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(fd2v.doc_vecs)

ValueError: setting an array element with a sequence.

In [10]:
fd2v.fit_knn_docs()

ValueError: setting an array element with a sequence.

In [29]:
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(fd2v.doc_vecs)

In [70]:
distances, indices = nbrs.kneighbors(fd2v.get_document_vector(input_).reshape(1,-1))

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)


In [71]:
indices[0]

array([292721, 105355, 197787,  82694, 200238,  52321, 387883, 150417,
        53783, 388527])

In [81]:
jamo_to_word_sent(petitions_text[388527])

'정부에서 갑자기 고졸 채용을 크게 늘린다고 발표했다 고졸 채용을 늘리는 것은 찬성한다 하지만 지금 대학졸업하고 일자리 없어 취급하지 못하는 청년들은 일자리가 더 줄어들게되어 영원한 실업자로 인생 낙오자로 살아가게 될 것 같다 너무나 힘들어하는 구직 중인 실업 청년들이 많다 정말 취업이 어려운 현실이다 취업 전선에서 번번히 낙오하여 갈 곳이 없는데 갑자기 지금 고졸 대 초반 채용을 늘리게되면 대졸 취업 못하고 있는 대 후반 초반 청년들은 어디를 가야하나'

In [20]:
a = np.array([1,2,3])
print(a is None)

False


In [12]:
k = k_means(cluster_num = 200)

TypeError: 'module' object is not callable

In [10]:
# 기존 append 효율 - 113
# 배열 기반 116
fd2v

array([[ 0.00024638,  0.00070633,  0.00331426, ...,  0.00128264,
         0.00076969,  0.00116821],
       [-0.01073391,  0.00116468, -0.0111112 , ...,  0.0033713 ,
        -0.00359978, -0.00536017],
       [-0.00704781,  0.00994899, -0.00013232, ...,  0.00082862,
         0.00282535, -0.00740313],
       ...,
       [-0.00597499, -0.0028156 ,  0.00338417, ...,  0.00553034,
        -0.0001105 , -0.0208315 ],
       [ 0.0068624 , -0.01209042, -0.00048188, ..., -0.0082753 ,
        -0.0181437 ,  0.00502233],
       [-0.00126394, -0.00831294, -0.00794914, ..., -0.00356445,
        -0.00565325, -0.01857061]], dtype=float32)

In [92]:
mecab = Mecab()

In [3]:
from petitions_dataset import fetch
fetch()

KeyboardInterrupt: Interrupted by user

In [5]:
from soynlp.hangle import decompose,compose

doublespace_pattern = re.compile('\s+')

korean_pattern = re.compile('[가-힣]')

english_pattern = re.compile('[a-zA-Z]')

def is_korean(char):
    if korean_pattern.match(char):
        return True
    else:
        return False
    
def is_english(char):
    if english_pattern.match(char):
        return True
    else:
        return False

def sentence_to_jamo(sent):

    def transform(char):
        if char == ' ':
            return char
        # 자모로 자르는 데 문제는 영어일 경우
        if(is_korean(char)):
            cjj = decompose(char)
        elif(is_english(char)):
            cjj = (char)
        else:
            return (' ')
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
        return cjj_
    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
sentence_to_jamo('어이고ㅗㅋaaf 켁켁 아이고오aaaaa未')
# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'

'ㅇㅓ-ㅇㅣ-ㄱㅗ- aaf ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-aaaaa '

In [6]:
korean_jamo_pattern = re.compile('[ㄱ-ㅎㅏ-ㅠ/-]')

def jamo_to_word(jamo):
  # idx를 기반으로 순차접근 방식으로 접근하면서 저는 소스가 흐트러졌는데 깔끔합니다.
    jamo_list, idx = [], 0
    word = ""
    while idx < len(jamo):
        if not is_jamo_korean(jamo[idx]):
            jamo_list.append(jamo[idx])
            idx += 1
        else:
            jamo_list.append(jamo[idx:idx + 3])
            idx += 3
            word = ""
    for jamo_char in jamo_list:
        if len(jamo_char) == 1:
            word += jamo_char
        elif jamo_char[2] == "-":
            word += compose(jamo_char[0], jamo_char[1], " ")
        else:
            word += compose(jamo_char[0], jamo_char[1], jamo_char[2])
    return word


In [7]:
korean_jamo_pattern = re.compile('[ㄱ-ㅎㅏ-ㅠ/-]')

def is_jamo_korean(char):
    if korean_jamo_pattern.match(char):
        return True
    else:
        return False
    
def jamo_to_word_sent(sent_jamo):
    sent = ""
    idx = 0
    while idx < len(sent_jamo)-1:
#        print(sent_jamo[idx:idx+3])
#        print(is_jamo_korean(sent_jamo[idx]))
        if(is_jamo_korean(sent_jamo[idx])) :
            sent+=jamo_to_word(sent_jamo[idx:idx+3])
            idx = idx+3
        else:
            sent+=jamo_to_word(sent_jamo[idx:idx+1])
            idx = idx+1
    return sent
    

In [64]:
jamo_to_word('ㅇㅓ-')

'어'

In [65]:
jamo_to_word_sent("ㅇㅓ-ㅇㅣ-ㄱㅗ- aaf ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-aaaaa")

어
어이
어이고
어이고 
어이고 a
어이고 aa
어이고 aaf
어이고 aaf 
어이고 aaf 켁
어이고 aaf 켁켁
어이고 aaf 켁켁 
어이고 aaf 켁켁 아
어이고 aaf 켁켁 아이
어이고 aaf 켁켁 아이고
어이고 aaf 켁켁 아이고오
어이고 aaf 켁켁 아이고오a
어이고 aaf 켁켁 아이고오aa
어이고 aaf 켁켁 아이고오aaa
어이고 aaf 켁켁 아이고오aaaa


'어이고 aaf 켁켁 아이고오aaaa'

In [55]:
def jamo_to_sentence(sent):
    ''.join([jamo_to_word(jamo) for jamo in sent])

In [9]:
mean_pos = ['NNG','NNP','VV','VA','VX','VCP','VCN','MAG','XPN','XSN','XSV','XSA','XR','SL']

petitions_jamo = []

for idx,text in enumerate(petitions_text):
    petitions_jamo.append(sentence_to_jamo(text))
    
#petitions_text = np.array(petitions_text)

In [10]:
a = pd.DataFrame()
a['text'] = petitions_text
a.to_csv('./petition_text.txt')

In [12]:
model = fasttext.train_unsupervised('./petition_text.txt')

In [8]:
model = fasttext.load_model('./petition_model.bin')

In [14]:
model.get_word_vector('가').shape

(100,)

In [63]:
input_ = "공공기관 비정규직 직원 정규직 변환"
input_ = sentence_to_jamo(input_).split(' ')

In [88]:
print(input_[4])
print(jamo_to_word(input_[4]))
print([jamo_to_word(word) for score , word in model.get_nearest_neighbors(input_[4])])

ㅂㅕㄴㅎㅘㄴ
변환
['변환기', '버전', '알파벳', '부호', '한자어', '변화없이', '자음', '표음', '주음', '복모음']


In [65]:
print(input_[0])
jamo_to_word(input_[0])

ㄱㅗㅇㄱㅗㅇㄱㅣ-ㄱㅘㄴ


'공공기관'

In [66]:
input_vector = np.array([model.get_word_vector(x) for x in input_]).mean(axis=0).reshape(-1,100)

In [17]:
%%time
all_vector = []
for i in range(len(petitions_text)):
    vector = np.array([model.get_word_vector(x) for x in petitions_text[i].split(' ')]).mean(axis=0)
    all_vector.append(vector)
all_vector = all_vector

CPU times: user 1min 16s, sys: 147 ms, total: 1min 16s
Wall time: 1min 16s


In [16]:
%%time
all_vector = np.empty((0,100),float)
for i in range(len(petitions_text)):
    if(i % 1000 == 0):
        print(i)
    vector = np.array([model.get_word_vector(x) for x in petitions_text[i].split(' ')]).mean(axis=0)
    all_vector = np.vstack((all_vector,vector))

0
1000


KeyboardInterrupt: 

In [94]:
pairwise_distances(all_vector[:2],all_vector[:40])
# 내가 하고 싶은 것
# 문장 내 단어 벡터를 모두 가져와서 (1 x word_num x dim)
# 분석하고자 하는 문장들과 비교 (N x word_num x dim)
# word_num은 문장마다 다르다.
# 단어와 단어의 거리를 비교하는 것 -> fasttext를 통한 거리
# 한 단어와 
# [input_docs,target_docs]

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


array([[0.0000000e+00, 6.5549225e-01, 5.7088828e-01, 6.3130307e-01,
        5.9144461e-01, 3.6297286e-01, 2.9952046e-01, 5.6083566e-01,
        5.8386767e-01, 5.5158788e-01, 4.6786976e-01, 6.7797214e-01,
        6.3912112e-01, 4.6483845e-01, 5.0252682e-01, 5.3230172e-01,
        5.3230172e-01, 5.3319240e-01, 4.7177288e-01, 5.2176178e-01,
        4.9080673e-01, 4.9080673e-01, 4.8306745e-01, 5.1160699e-01,
        3.9521903e-01, 2.9303470e-01, 6.1914271e-01, 5.2764261e-01,
        6.1598593e-01, 6.5073919e-01, 5.8448327e-01, 5.8954298e-01,
        7.0592719e-01, 4.9236202e-01, 5.7352358e-01, 5.3898257e-01,
        4.1556561e-01, 5.4808295e-01, 4.7958609e-01, 3.6664855e-01],
       [6.5549225e-01, 2.9802322e-08, 6.6742945e-01, 7.2424084e-01,
        5.6546879e-01, 5.9247434e-01, 5.4293776e-01, 6.7454827e-01,
        6.6578710e-01, 5.7573110e-01, 7.1565264e-01, 6.7311478e-01,
        6.7900997e-01, 7.1363282e-01, 6.4975703e-01, 6.1589456e-01,
        6.1589456e-01, 5.9961027e-01, 6.0895228

In [93]:
for i in range(10000000):
    if(i % 100000 == 0):
        print(i)
    pairwise_distances(all_vector[:2],all_vector[:40])

0


/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

...

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:323: RuntimeWa

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [67]:
# 이걸 아이디 기반으로 구현 가능?
dist = pairwise_distances(input_vector,all_vector,metric='cosine').reshape(-1)
dist_rank = dist.argsort()
ranks = np.empty_like(dist_rank)
ranks[dist_rank] = np.arange(len(dist))
#ranks = ranks[::-1]

/Users/nogyeongju/opt/anaconda3/envs/lion/lib/python3.8/site-packages/sklearn/utils/extmath.py:77: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norms, norms)


In [68]:
ranks[0]

1606

In [77]:
np.where(dist_rank > len(dist_rank) - 10)[0]

array([ 1192,  6167, 15930, 34078, 39227, 55799, 65037, 70252, 81715])

In [ ]:
model.

In [82]:
jamo_to_word_sent(petitions_text[65037])

'현재 여만명의 청년실업자가 단지 취업하지 못했다는 이유로 가족과 사회적으로 심리적으로 위축이 심하며 궁핍한 생활수준입니다 기본적인 삶이 유지될 수 있도록 미취업청년복지카드발급를 제안합니다 청년실업자 인당 무이자로 월 만원한도로 년간 취업시 제한 자유로이 사용하게 하며 년 이후부터 년까지 차등적 이자를 부담하게 하여 상환하도록 합니다 금융기관이 취급하며 정부가 위탁수수료 및 지급보증을 하면 됩니다 년 소요금액 만명 만원 조 천억 정부부담비용 년 조 천억 억 예전에 복지부의 바우처카드결제기능 위탁수수료 연평균 효과 청년들의 심리불안해소로 안정적 취업의 기회를 가질 수 있으며 또한 청년들의 소비성향이 이기 때문에 경제 활성화에 큰 도움이 된다 그리고 청년세대보다 신용도가 높은 세대는 없습니다'

In [89]:
all_vector

NameError: name 'all_vector' is not defined

In [108]:
dist[ranks<100]

array([0.43171257, 0.44999635, 0.3865162 , 0.44935232, 0.42418385,
       0.45018113, 0.42383975, 0.44990152, 0.36914718, 0.43044013,
       0.43423414, 0.4372871 , 0.4483173 , 0.42273247, 0.4434904 ,
       0.44616628, 0.43913156, 0.44868279, 0.43211436, 0.44153595,
       0.43033016, 0.43413532, 0.43027818, 0.37053096, 0.37344015,
       0.40837693, 0.41225243, 0.44263726, 0.43274927, 0.42251527,
       0.44138205, 0.43494797, 0.42382437, 0.42712355, 0.42387867,
       0.39659047, 0.4386269 , 0.44581085, 0.44288886, 0.43714178,
       0.36957502, 0.41929793, 0.41099143, 0.42093515, 0.39009094,
       0.365658  , 0.37506127, 0.44503212, 0.44876552, 0.4307148 ,
       0.42609608, 0.42883408, 0.40387452, 0.40824723, 0.43925738,
       0.39907634, 0.3987413 , 0.40429705, 0.38640487, 0.4066025 ,
       0.43366218, 0.43278384, 0.43054378, 0.43373537, 0.39008904,
       0.4280058 , 0.44939363, 0.44226384, 0.43498087, 0.43011963,
       0.4233204 , 0.44560683, 0.42227775, 0.42227775, 0.42571

In [109]:
dist[ranks>149300]

array([1.3164655, 1.3424485, 1.3551731, 1.3475543, 1.3348835, 1.3433473,
       1.3471396, 1.3401057, 1.3301588, 1.3688873, 1.3424485, 1.3202717,
       1.4452196, 1.3380278, 1.3204883, 1.3257976, 1.3254621, 1.3310657,
       1.3627391, 1.3233595, 1.3493037, 1.3174868, 1.3373132, 1.315732 ,
       1.3321595, 1.3321595, 1.3321595, 1.3829622, 1.3274319, 1.3820299,
       1.3785083, 1.3438466, 1.3197906, 1.3940107, 1.3275523, 1.3235788,
       1.3184813, 1.3182813, 1.4159698, 1.3424485, 1.3387625, 1.3253598,
       1.3540888, 1.3344212, 1.3179842, 1.322701 , 1.324781 , 1.3628055,
       1.3278427, 1.347405 , 1.3219624, 1.3568454, 1.3947802, 1.4210197,
       1.3424485, 1.4152815, 1.4183012, 1.3216035, 1.364991 , 1.320646 ,
       1.320646 , 1.3605459, 1.34044  , 1.3469236, 1.3238648, 1.34044  ,
       1.4558554, 1.3259153, 1.3653272, 1.327681 , 1.3349936, 1.3473029,
       1.368722 , 1.3920552, 1.3170159, 1.3340992, 1.4021859, 1.3723606,
       1.3694212, 1.326851 , 1.323587 , 1.4064823, 

In [110]:
petitions_text[ranks<10]

array(['안녕하세요. 저는 4남매를 키우는 주부입니다. 12세 10세 7세 4세. 아이들 연령대가 다양하죠. 오늘아침에도 저출산 문제에 관한 기사를 읽으며 답답함에 한숨이 나옵니다. 일주일에 한번꼴로 저는 국가에서 지원하는 다자녀 혜택, 근로장려금, 자녀장려금등 여러가지 제도를 검색하지만 현실적으로는 제대로된 도움을 받지 못합니다. 자녀장려금 과 근로장려금 같은 경우는 부부 맞벌이 합산 소득이 문제가 되지요. 또한 재산도 문제가 됩니다. 재산이 많아서는 결코 아닙니다. 3년전 월세를 감당하는것이 힘들어 국민주택 대출로 2.7%금리로 1억4천3백만원 대출을 끼고 아파트를 장만한것이 문제입니다. 버리는돈 아까워 내집이라도 만들어보자고 했던것이 재산이라며 발목을 잡네요. 현재 아파트에 현관만큼도 지분을 가지지못했더군요. 3년동안 갚은이자가 천만원이 조금넘고 원금이 8백정도 갚앞으니 이제 잊은 1억3천이 조금더 남았네요. 장려금 기준 주택은 공시시가로 1억입니다. 그리고 부부합산소득은 어쩌겠어요. 대출금에 이자에 생활비에 아이들 교육비에 끝도없이 들어가는 돈을 충당하려 남편은 4대보험이 되는 공장을 그만두고 송전탑 전기 보수일을 하고 있습니다. 송전탑 아시죠? 산꼭대기에 보면 서있는 전기철탑이예요. 전류가 얼마나 흐르는지 아세요? 여차하면 사망입니다. 하루일당 20 만원에 목숨을 내놓고 일을합니다. 비오면 무조건 쉬고 한전에서 휴전을 못하면 쉬고. 여름 겨울 에어컨 히터로 전력량이 많은계절은 일을 못해쉬고,해서 연봉으로는 4천이 조금 안되네요. 그런데 햇살론 혜택도 못받아요. 이유는 일용직이더라도 3개월간 같은회사 이름으로 급여가 들어와야 한대요. 저희남편이 전국을 돌며 일을하는 회사. 저희 남편을 일꾼으로 써주는 회사는 세군데 정도 됩니다. 공사지역이 다르니 그럴수밖에요. 헌데 정부지원 서민대출 혜택도 받을수가 없네요. 말은 서민지원이나 4대보험이 안되면 90%이상이 대출거절이예요. 저도 맞벌이를 해보았지만 아이들을 맡기는 비용이 더드네요. 셋째아이같은경우 일

In [42]:
petitions_text = np.array(petitions_text)

In [128]:
petitions_text.shape

(149450,)

In [112]:
dist_rank.shape

(1, 149450)

In [107]:
ranks = np.empty_like(dist_rank)

In [81]:
np.array(petitions_text[ranks<1000])[641]

'가상화폐 거래소 폐소 반대 합니다'

In [137]:
type(model)

fasttext.FastText._FastText